In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima
from statsmodels.tsa.api import VAR
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from pygam import GAM, s, LinearGAM
import pickle
import xgboost as xgb
from xgboost import XGBRegressor
from prophet import Prophet

In [ ]:
df_train = pd.read_csv("../Daten/validation_data.csv")

In [ ]:
df_train['Date and time'] = pd.to_datetime(df_train['Date and time'], format='mixed', dayfirst=True, errors='coerce')
df_train.set_index('Date and time', inplace=True)

In [ ]:
df_val = pd.read_csv("../Daten/test_data.csv")

In [ ]:
df_val['Date and time'] = pd.to_datetime(df_val['Date and time'], format='mixed', dayfirst=True, errors='coerce')
df_val.set_index('Date and time', inplace=True)

# ARIMA

In [ ]:
auto_arima_model = auto_arima(df_train["Power (kW)"].iloc[-144:], seasonal=False, trace=True, suppress_warnings=True, stepwise=False, m=1)
p, d, q = auto_arima_model.order

print(auto_arima_model.summary)

In [ ]:
arima_fit = auto_arima_model.fit(df_train["Power (kW)"].iloc[-144:])

In [ ]:
def check_stationarity(series):
    result = adfuller(series)
    print(f"p-Wert: {result[1]} (stationär, wenn < 0.05)")

check_stationarity(df_train["Power (kW)"].iloc[-144:])

In [ ]:
arima = ARIMA(df_train["Power (kW)"].iloc[-144:], order=auto_arima_model.order)
arima_fit = arima.fit()

In [ ]:
forecasts = arima_fit.get_forecast(steps=144)

In [ ]:
actual_values = df_val["Power (kW)"].iloc[:144]

mae = mean_absolute_error(actual_values, forecasts.predicted_mean)
rmse = root_mean_squared_error(actual_values, forecasts.predicted_mean)

print("Vorhersagen:", forecasts.predicted_mean)
print("Tatsächliche Werte:", actual_values.values)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
horizon = 144

plt.figure(figsize=(12, 6))
plt.title("ARIMA Vorhersage")
plt.plot(df_val.index[:horizon], df_val["Power (kW)"].iloc[:horizon], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:horizon], forecasts.predicted_mean[:horizon], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

In [ ]:
forecasts = []

# Rollierende Vorhersagen
current_train = df_train["Power (kW)"].copy()

for i in range(0, 144, 6):
    # Modell fitten
    model = ARIMA(current_train, order=auto_arima_model.order)
    model_fit = model.fit()

    # Vorhersage für 6 Schritte
    forecast = model_fit.forecast(steps=6)
    forecasts.extend(forecast)

    # Tatsächliche Werte der nächsten 6 Schritte hinzufügen
    actual_values = df_val["Power (kW)"].iloc[i:i + 6]
    current_train = pd.concat([current_train, actual_values])  # Aktualisieren
# Ergebnisse als DataFrame speichern
forecast_results = pd.DataFrame({
    'Forecast': forecasts[:len(df_val["Power (kW)"])],
    'Actual': df_val["Power (kW)"].iloc[:144].values.flatten()
}, index=df_val["Power (kW)"].index[:144])

print(forecast_results)

In [ ]:
forecast_results = pd.DataFrame({
    'Forecast': forecasts[:len(df_val["Power (kW)"])],
    'Actual': df_val["Power (kW)"].iloc[:144].values.flatten()
}, index=df_val["Power (kW)"].index[:144])

print(forecast_results)

In [ ]:
import pandas as pd
import numpy as np

# Annahme: forecast_results und df_val sind DataFrames mit den entsprechenden Spalten
forecast_values = forecast_results["Forecast"].values
actual_values = df_val["Power (kW)"].values

# Anzahl der Schritte pro Vorhersage
steps_per_forecast = 6

# Aufteilen der Vorhersagen und tatsächlichen Werte in 6er-Gruppen
forecast_groups = np.split(forecast_values, len(forecast_values) // steps_per_forecast)
actual_groups = np.split(actual_values[:len(forecast_values)], len(forecast_values) // steps_per_forecast)

# Funktion zur Berechnung der Metriken für jede Gruppe
def calculate_metrics(actual, predicted):
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mad = np.median(np.abs(actual - predicted))
    smape = 100 * np.mean(
        np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) / 2)
    )
    me = np.mean(actual - predicted)
    return rmse, mad, smape, me

# Metriken für jede Gruppe berechnen
metrics = [calculate_metrics(actual, forecast) for actual, forecast in zip(actual_groups, forecast_groups)]

# Ergebnisse in ein DataFrame umwandeln
metrics_df = pd.DataFrame(metrics, columns=["RMSE", "MAD", "sMAPE (%)", "Bias"])

# Durchschnittswerte aller Gruppen berechnen
average_metrics = metrics_df.mean()

# Ergebnisse ausgeben
print("Durchschnittliche Metriken über alle Gruppen:")
print(average_metrics)


In [ ]:
horizon = 144

plt.figure(figsize=(12, 6))
plt.title("ARIMA Vorhersage")
plt.plot(df_val.index[:horizon], df_val["Power (kW)"].iloc[:horizon], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:horizon], forecast_results["Forecast"], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

In [ ]:
arima_forecasts = forecast_results["Forecast"]

## SARIMAX Forecast

In [ ]:
auto_arima_model.order

In [ ]:
sarima_modell = SARIMAX(df_train["Power (kW)"].iloc[-144:], order=auto_arima_model.order, seasonal_order=(1,1,1,6))

In [ ]:
fit = sarima_modell.fit()

In [ ]:
forecasts = fit.get_forecast(steps=144)

In [ ]:
forecasts.predicted_mean

In [ ]:
import pandas as pd
import numpy as np

# Annahme: forecast_results und df_val sind DataFrames mit den entsprechenden Spalten
forecast_values = forecasts.predicted_mean
actual_values = df_val["Power (kW)"].iloc[:144].values

# Anzahl der Schritte pro Vorhersage
steps_per_forecast = 6

# Aufteilen der Vorhersagen und tatsächlichen Werte in 6er-Gruppen
forecast_groups = np.split(forecast_values, len(forecast_values) // steps_per_forecast)
actual_groups = np.split(actual_values[:len(forecast_values)], len(forecast_values) // steps_per_forecast)

# Funktion zur Berechnung der Metriken für jede Gruppe
def calculate_metrics(actual, predicted):
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mad = np.median(np.abs(actual - predicted))
    smape = 100 * np.mean(
        np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) / 2)
    )
    me = np.mean(actual - predicted)
    return rmse, mad, smape, me

# Metriken für jede Gruppe berechnen
metrics = [calculate_metrics(actual, forecast) for actual, forecast in zip(actual_groups, forecast_groups)]

# Ergebnisse in ein DataFrame umwandeln
metrics_df = pd.DataFrame(metrics, columns=["RMSE", "MAD", "sMAPE (%)", "Bias"])

# Durchschnittswerte aller Gruppen berechnen
average_metrics = metrics_df.mean()

# Ergebnisse ausgeben
print("Durchschnittliche Metriken über alle Gruppen:")
print(average_metrics)


In [ ]:
actual_values = df_val["Power (kW)"].iloc[:144]

mae = mean_absolute_error(actual_values, forecasts.predicted_mean)
rmse = root_mean_squared_error(actual_values, forecasts.predicted_mean)

print("Vorhersagen:", forecasts.predicted_mean)
print("Tatsächliche Werte:", actual_values.values)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
plt.figure(figsize=(12, 6))
plt.title("SARIMA Vorhersage")
plt.plot(df_val.index[:144], df_val["Power (kW)"].iloc[:144], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:144], forecasts.predicted_mean[:144], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

# Prophet

In [ ]:
df_prophet = df_train.reset_index()[["Date and time", "Power (kW)"]]
df_prophet.columns = ["ds", "y"]
df_prophet_val = df_val.reset_index()[["Date and time", "Power (kW)"]]
df_prophet_val.columns = ["ds", "y"]

In [ ]:
df_prophet

In [ ]:
train_size = 144
forecast_size = 144

In [ ]:
train_set = df_prophet
val_set = df_prophet_val

In [ ]:
from prophet import Prophet

prophet = Prophet(
    changepoint_prior_scale= 0.01, 
    seasonality_prior_scale= 0.5,
    weekly_seasonality=True,
    daily_seasonality=True,
    yearly_seasonality=False
)

In [ ]:
prophet.fit(train_set)

In [ ]:
future = pd.DataFrame({"ds": val_set["ds"].iloc[:144]})

In [ ]:
future

In [ ]:
forecast = prophet.predict(future)

In [ ]:
fig = prophet.plot_components(forecast)

In [ ]:
forecast

In [ ]:
actuals = val_set['y'].iloc[:144]
predictions = forecast['yhat'].iloc[:144]

prophet_mse = mean_squared_error(actuals, predictions)
prophet_mae = mean_absolute_error(actuals, predictions)
prophet_rmse = root_mean_squared_error(actuals, predictions)

print(f'Mean Squared Error: {prophet_mse}')
print(f'Mean Absolute Error: {prophet_mae}')
print(f'Root Mean Squared Error: {prophet_rmse}')

In [ ]:
predictions

In [ ]:
plt.figure(figsize=(12, 6))
plt.title(label="Prophet Vorhersage")
plt.plot(val_set['ds'].iloc[:144], val_set['y'].iloc[:144], label="Validierungsdaten")
plt.plot(val_set['ds'].iloc[:144], predictions.iloc[:144], label="Vorhersage")
plt.ylabel("Power (kW)")
plt.xlabel("Time")
plt.legend()
plt.show()

In [ ]:
import pandas as pd

# Liste für Vorhersagen
forecasts = []

# Initiale Trainingsdaten
current_train = train_set[["ds", "y"]].iloc[-144:].copy()

for i in range(0, 144, 6):
    # Prophet-Modell initialisieren
    model = Prophet(
        changepoint_prior_scale= 0.01,
        seasonality_prior_scale= 0.5,
        weekly_seasonality=True,
        daily_seasonality=True,
        yearly_seasonality=False
    )
    model.fit(current_train)

    # DataFrame für Vorhersagen erstellen
    future = pd.DataFrame({
        "ds": val_set["ds"].iloc[i:i + 6]
    })

    # Vorhersage für 6 Schritte
    forecast = model.predict(future)
    forecasts.extend(forecast["yhat"].values)

    # Tatsächliche Werte der nächsten 6 Schritte hinzufügen
    actual_values = val_set[["ds", "y"]].iloc[i:i + 6]
    current_train = pd.concat([current_train, actual_values])  # Aktualisieren

# Ergebnisse als DataFrame speichern
forecast_results = pd.DataFrame({
    "Forecast": forecasts[:len(val_set["y"])],
    "Actual": val_set["y"].iloc[:144].values.flatten()
}, index=val_set["ds"].iloc[:144])

print(forecast_results)

In [ ]:
import pandas as pd
import numpy as np

# Annahme: forecast_results und df_val sind DataFrames mit den entsprechenden Spalten
forecast_values = forecast_results["Forecast"].values
actual_values = df_val["Power (kW)"].values

# Anzahl der Schritte pro Vorhersage
steps_per_forecast = 6

# Aufteilen der Vorhersagen und tatsächlichen Werte in 6er-Gruppen
forecast_groups = np.split(forecast_values, len(forecast_values) // steps_per_forecast)
actual_groups = np.split(actual_values[:len(forecast_values)], len(forecast_values) // steps_per_forecast)

# Funktion zur Berechnung der Metriken für jede Gruppe
def calculate_metrics(actual, predicted):
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mad = np.median(np.abs(actual - predicted))
    smape = 100 * np.mean(
        np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) / 2)
    )
    me = np.mean(actual - predicted)
    return rmse, mad, smape, me

# Metriken für jede Gruppe berechnen
metrics = [calculate_metrics(actual, forecast) for actual, forecast in zip(actual_groups, forecast_groups)]

# Ergebnisse in ein DataFrame umwandeln
metrics_df = pd.DataFrame(metrics, columns=["RMSE", "MAD", "sMAPE (%)", "Bias"])

# Durchschnittswerte aller Gruppen berechnen
average_metrics = metrics_df.mean()

# Ergebnisse ausgeben
print("Durchschnittliche Metriken über alle Gruppen:")
print(average_metrics)


In [ ]:
horizon = 144

plt.figure(figsize=(12, 6))
plt.title("Prophet Vorhersage")
plt.plot(df_val.index[:horizon], df_val["Power (kW)"].iloc[:horizon], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:horizon], forecast_results["Forecast"], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

In [ ]:
prophet_forecasts = forecast_results["Forecast"]

# ML

In [ ]:
def create_features(df):
    df = df.copy()
    df["hour"] = df.index.hour
    df["dayofweek"] = df.index.dayofweek
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["dayofyear"] = df.index.dayofyear

    return df

In [ ]:
def add_lags(df):
    # Sicherstellen, dass die Zielspalte existiert
    if "Power (kW)" not in df.columns:
        raise ValueError("Die Spalte 'Power (kW)' muss im DataFrame enthalten sein.")
    
    # Lag-Features iterativ hinzufügen
    for lag_steps in [6, 12, 18]:
        lag_column = f"lag_{lag_steps}"  # Eindeutiger Name für jede Lag-Spalte
        df[lag_column] = df["Power (kW)"].shift(lag_steps)  # Werte verschieben
    
    return df 

In [ ]:
train = create_features(df_train)
val = create_features(df_val)

trainval = pd.concat([train, val])

In [ ]:
train.columns
trainval = add_lags(trainval)

In [ ]:
print(len(train))
print(len(val))

In [ ]:
train = trainval[:52704]
val = trainval[52704:]


In [ ]:
train.drop(columns=['Density adjusted wind speed (m/s)', 'Wind direction (°)',
       'Nacelle position (°)', 'Wind speed (m/s)'])
val.drop(columns=['Density adjusted wind speed (m/s)', 'Wind direction (°)',
       'Nacelle position (°)', 'Wind speed (m/s)'])


In [ ]:
FEATURES = ['hour', 'dayofweek', 'month',
       'year', 'dayofyear', 'lag_6', 'lag_12', 'lag_18']
TARGET = ['Power (kW)']


In [ ]:
X_train = train[FEATURES]
y_train = train[TARGET]

X_val = val[FEATURES]
y_val = val[TARGET]


## XGBoost

In [ ]:
with open('xgb_model.pkl', 'rb') as file:
    xgb = pickle.load(file)

In [ ]:
xgb = XGBRegressor(
    objective='reg:squarederror',
    max_depth=3,
    learning_rate=0.05,
    n_estimators=100,
    gamma=0.5,
    min_child_weight=10,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=46.41588833612773,
    reg_lambda=2.154434690031882,
    random_state=42
)

In [ ]:
xgb_fit = xgb.fit(X_train, y_train,
        verbose=True)

### Feature Importances

In [ ]:
## Feature Importance
fi = pd.DataFrame(data=xgb_fit.feature_importances_, index=xgb_fit.feature_names_in_,columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
## Forecast on Validationset


In [ ]:

val['prediction'] = xgb_fit.predict(X_val)
df_val.merge(val[['prediction']], how='left', left_index=True, right_index=True)


In [ ]:
import pandas as pd
import numpy as np

# Annahme: forecast_results und df_val sind DataFrames mit den entsprechenden Spalten
forecast_values = val['prediction'].values
actual_values = df_val["Power (kW)"].values

# Anzahl der Schritte pro Vorhersage
steps_per_forecast = 6

# Aufteilen der Vorhersagen und tatsächlichen Werte in 6er-Gruppen
forecast_groups = np.split(forecast_values, len(forecast_values) // steps_per_forecast)
actual_groups = np.split(actual_values[:len(forecast_values)], len(forecast_values) // steps_per_forecast)

# Funktion zur Berechnung der Metriken für jede Gruppe
def calculate_metrics(actual, predicted):
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mad = np.median(np.abs(actual - predicted))
    smape = 100 * np.mean(
        np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) / 2)
    )
    me = np.mean(actual - predicted)
    return rmse, mad, smape, me

# Metriken für jede Gruppe berechnen
metrics = [calculate_metrics(actual, forecast) for actual, forecast in zip(actual_groups, forecast_groups)]

# Ergebnisse in ein DataFrame umwandeln
metrics_df = pd.DataFrame(metrics, columns=["RMSE", "MAD", "sMAPE (%)", "Bias"])

# Durchschnittswerte aller Gruppen berechnen
average_metrics = metrics_df.mean()

# Ergebnisse ausgeben
print("Durchschnittliche Metriken über alle Gruppen:")
print(average_metrics)


In [ ]:
actual_values = df_val["Power (kW)"].iloc[:144]

mae = mean_absolute_error(actual_values, val['prediction'].iloc[:144])
rmse = root_mean_squared_error(actual_values, val['prediction'].iloc[:144])

print("Vorhersagen:", val['prediction'].iloc[:144])
print("Tatsächliche Werte:", actual_values.values)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
horizon = 144

plt.figure(figsize=(10, 6))
plt.title("XGBoost Vorhersage")
plt.plot(df_val.index[:horizon], df_val["Power (kW)"].iloc[:horizon], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:horizon], val['prediction'].iloc[:horizon], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

In [ ]:
xgb_forecasts = val['prediction'].iloc[:144]

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(
    max_depth=10,
    max_features='sqrt',
    min_samples_leaf=4,
    min_samples_split=2,
    n_estimators=200,
    random_state=42
)

# Das Modell ist bereit zur Verwendung (z. B. fit() auf Trainingsdaten)

In [ ]:
rf.fit(X_train.iloc[:], y_train.iloc[:])

In [ ]:
print("Trainingsdaten:", X_train.shape)
print("Validierungsdaten:", X_val.shape)

In [ ]:
val['prediction'] = rf.predict(X_val)
df_val.merge(val[['prediction']], how='left', left_index=True, right_index=True)

In [ ]:
import pandas as pd
import numpy as np

# Annahme: forecast_results und df_val sind DataFrames mit den entsprechenden Spalten
forecast_values = val['prediction'].values
actual_values = df_val["Power (kW)"].values

# Anzahl der Schritte pro Vorhersage
steps_per_forecast = 6

# Aufteilen der Vorhersagen und tatsächlichen Werte in 6er-Gruppen
forecast_groups = np.split(forecast_values, len(forecast_values) // steps_per_forecast)
actual_groups = np.split(actual_values[:len(forecast_values)], len(forecast_values) // steps_per_forecast)

# Funktion zur Berechnung der Metriken für jede Gruppe
def calculate_metrics(actual, predicted):
    rmse = np.sqrt(np.mean((actual - predicted) ** 2))
    mad = np.median(np.abs(actual - predicted))
    smape = 100 * np.mean(
        np.abs(actual - predicted) / ((np.abs(actual) + np.abs(predicted)) / 2)
    )
    me = np.mean(actual - predicted)
    return rmse, mad, smape, me

# Metriken für jede Gruppe berechnen
metrics = [calculate_metrics(actual, forecast) for actual, forecast in zip(actual_groups, forecast_groups)]

# Ergebnisse in ein DataFrame umwandeln
metrics_df = pd.DataFrame(metrics, columns=["RMSE", "MAD", "sMAPE (%)", "Bias"])

# Durchschnittswerte aller Gruppen berechnen
average_metrics = metrics_df.mean()

# Ergebnisse ausgeben
print("Durchschnittliche Metriken über alle Gruppen:")
print(average_metrics)


In [ ]:

rmse = np.sqrt(np.mean((actual_values - val['prediction']) ** 2))

# Berechnung von MAD
mad = np.median(np.abs(actual_values - val['prediction']))

# Berechnung von sMAPE
smape = 100 * np.mean(
    np.abs(actual_values - val['prediction']) / ((np.abs(actual_values) + np.abs(val['prediction'])) / 2)
)
me = np.mean(actual_values - val['prediction'])

# Ergebnisse ausgeben
print("RMSE:", rmse)
print("MAD:", mad)
print("sMAPE:", smape, "%")
print("ME:", me)

In [ ]:
horizon = 144

plt.figure(figsize=(10, 6))
plt.title("Random Forest Vorhersage")
plt.plot(df_val.index[:horizon], df_val["Power (kW)"].iloc[:horizon], label="Tatsächliche Werte (Validierung)")
plt.plot(df_val.index[:horizon], val['prediction'].iloc[:horizon], label="Vorhersagen")
plt.ylabel("Power (kW)")
plt.xlabel("Datum")
plt.legend()
plt.show()

In [ ]:
rf_forecasts = val['prediction'].iloc[:144]

In [ ]:
print(df_val.index[start:horizon][xticks_positions])  # Prüfen, ob die Labels existieren

In [ ]:
horizon = 144
start = 0

xticks_positions = range(0, horizon, horizon // 10)  # Jeder zweite Wert oder nach Bedarf
print(xticks_positions)


plt.figure(figsize=(10, 6))
plt.title("Forecasts aller Modelle im Vergleich")
plt.plot(df_val.index[start:horizon], df_val["Power (kW)"].iloc[:144], label="Tatsächliche Werte (Testdaten)", linewidth=2)
plt.plot(df_val.index[start:horizon], prophet_forecasts[start:horizon], label="Prophet", linewidth=2, alpha=0.8)
plt.plot(df_val.index[start:horizon], rf_forecasts[start:horizon], label="Random Forest", linewidth=2, alpha=0.8)
plt.plot(df_val.index[start:horizon], xgb_forecasts[start:horizon], label="XGBoost", linewidth=2, alpha=0.8)
plt.plot(df_val.index[start:horizon], arima_forecasts[start:horizon], label="ARIMA", linewidth=2, alpha=0.8)
plt.ylabel("Leisung (kW)")
plt.xlabel("Datum")
#plt.xticks(xticks_positions)  # xticks hinzufügen
plt.rcParams.update({'font.size': 14})
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
horizon = 144
start = 0

xticks_positions = range(0, horizon, horizon // 10)  # Jeder zweite Wert oder nach Bedarf
print(xticks_positions)


plt.figure(figsize=(10, 6))
plt.title("Forecasts von Random Forest und XGBoost im Vergleich")
plt.plot(df_val.index[start:horizon], df_val["Power (kW)"].iloc[:144], label="Tatsächliche Werte (Testdaten)",)
plt.plot(df_val.index[start:horizon], rf_forecasts[start:horizon], label="Random Forest",)
plt.plot(df_val.index[start:horizon], xgb_forecasts[start:horizon], label="XGBoost",)
plt.ylabel("Leisung (kW)")
plt.xlabel("Datum")
#plt.xticks(xticks_positions)  # xticks hinzufügen
plt.rcParams.update({'font.size': 14})
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()